In [1]:
import os, time, datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import logging
tqdm.pandas()
import seaborn as sns
from sklearn.model_selection import train_test_split

#NN Packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, random_split,DataLoader, RandomSampler, SequentialSampler

logger = logging.getLogger(__name__)


if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Processing text data

In [2]:
# a = pd.read_csv('/kaggle/input/textphase1/'+"data/catalog_english_taxonomy.tsv",sep="\t")
# a.head()

In [3]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

class SigirPreprocess():
    
    
    def __init__(self, text_data_path):
        self.text_data_path = text_data_path
        self.train = None
        self.dict_code_to_id = {}
        self.dict_id_to_code = {}
        self.list_tags = {}
        self.sentences = []
        self.labels = []
        self.text_col = None
        self.X_test = None
    
    
    def prepare_data(self ):
        
        #loading the train data and test data
        catalog_eng = pd.read_csv(self.text_data_path+"data/catalog_english_taxonomy.tsv",sep="\t")
        X_train= pd.read_csv(self.text_data_path+"data/X_train.tsv",sep="\t")
        Y_train= pd.read_csv(self.text_data_path+"data/Y_train.tsv",sep="\t")
        self.list_tags = list(Y_train['Prdtypecode'].unique())
        
        for i,tag in enumerate(self.list_tags):
            self.dict_code_to_id[tag] = i 
            self.dict_id_to_code[i]=tag
        
        #map 
        Y_train['labels']=Y_train['Prdtypecode'].map(self.dict_code_to_id)
        
        #merge the train
        train=pd.merge(left=X_train,right=Y_train,
               how='left',left_on=['Integer_id','Image_id','Product_id'],
               right_on=['Integer_id','Image_id','Product_id'])
        prod_map=pd.Series(catalog_eng['Top level category'].values,
                           index=catalog_eng['Prdtypecode']).to_dict()
        
        #creating the mapping
        train['product'] = train['Prdtypecode'].map(prod_map)
        train['title_len']=train['Title'].progress_apply(lambda x : len(x.split()) if pd.notna(x) else 0)
        train['desc_len']=train['Description'].progress_apply(lambda x : len(x.split()) if pd.notna(x) else 0)
        train['title_desc_len']=train['title_len'] + train['desc_len']
        train.loc[train['Description'].isnull(), 'Description'] = " "
        train['title_desc'] = train['Title'] + " " + train['Description']
        
        self.train = train
        
    def get_sentences(self, text_col, remove_null_rows=False):
        self.text_col = text_col
        if remove_null_rows==True:
            new_train = self.train[self.train[text_col].notnull()]

        else:
            new_train = self.train.copy()
            
        self.sentences = new_train[text_col].values
        self.labels = new_train['labels'].values
    
    def prepare_test(self, text_col):
        X_test=pd.read_csv(self.text_data_path+"data/x_test_task1_phase1.tsv",sep="\t")
        X_test.loc[X_test['Description'].isnull(), 'Description'] = " "
        X_test['title_desc'] = X_test['Title'] + " " + X_test['Description']
        self.X_test = X_test
        self.test_sentences = X_test[text_col].values
        

In [4]:
text_col = 'title_desc'
max_len = 256
val_size = 0.1

# model_str_dict = {'c':'camembert',
#                  'f':'flaubert'}
# # 'f' for flaubert & 'c' for camembert
# case='f' 
# model_str = model_str_dict[case]

In [5]:
Preprocess = SigirPreprocess("/kaggle/input/textphase1/")
Preprocess.prepare_data()
Preprocess.get_sentences(text_col, True)

100%|██████████| 84916/84916 [00:00<00:00, 89543.91it/s]


In [6]:
sentences = Preprocess.sentences
labels = Preprocess.labels
print("Total number of sentences:{}, labels:{}".format(len(sentences), len(labels)))

Total number of sentences:84916, labels:84916


In [7]:
# sns.countplot(x='product', data=self.train)
# sns.countplot(x='Prdtypecode', data=self.train)
# sns.distplot(Preprocess.train['title_len'])
# sns.distplot(Preprocess.train['title_desc_len'])
# np.percentile(Preprocess.train['title_desc_len'], 99)

In [8]:
len(Preprocess.dict_code_to_id)

27

In [9]:
from transformers import XLMForSequenceClassification
from transformers import FlaubertModel, FlaubertTokenizer,FlaubertForSequenceClassification,AdamW, FlaubertConfig 
from torch.nn import Dropout,Conv1d, Linear
from transformers.modeling_utils import SequenceSummary

In [10]:
# a1 = sentences[0]
# max_len = 40
# modelname = 'flaubert-base-cased'
# tokenizer = FlaubertTokenizer.from_pretrained(modelname, do_lowercase=False)

# encoded_dict = tokenizer.encode_plus(
#                             a1,                      # Sentence to encode.
#                             add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                             max_length = max_len,           # Pad & truncate all sentences.
#                             pad_to_max_length = True,
#                             return_attention_mask = True,   # Construct attn. masks.
#                             return_tensors = 'pt',     # Return pytorch tensors.
#                        )


# iid = encoded_dict['input_ids']
# mask = encoded_dict['attention_mask']

# iid,mask

# # modelname = 'flaubert-base-cased'

# model = CustFlaubertForSequenceClassification.from_pretrained(
#         modelname, # Use the 12-layer BERT model, with an uncased vocab.
#         # num_labels = len(Preprocess.dict_code_to_id), # The number of output labels--2 for binary classification.
#         # You can increase this for multi-class tasks.   
#         output_attentions = False, # Whether the model returns attentions weights.
#         output_hidden_states = False, # Whether the model returns all hidden-states.
# )

# outputs, embed1 = model(iid, token_type_ids=None, attention_mask=mask, 
# )

In [11]:
# #max length after tokenization
# _max_len = 0
# # For every sentence...
# for sent in tqdm(sentences):

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)

#     # Update the maximum sentence length.
#     _max_len = max(_max_len, len(input_ids))

# print('Max sentence length: ', _max_len)

In [12]:
modelname = 'flaubert-base-cased'
tokenizer = FlaubertTokenizer.from_pretrained(modelname, do_lowercase=False)

In [13]:
#function to prepare input for model training
def prep_input(sentences,labels, max_len):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in tqdm(sentences):
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_len,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    if labels is not None:
        labels = torch.tensor(labels)
        return input_ids,attention_masks,labels
    else:
        return input_ids,attention_masks
    

In [14]:
input_ids,attention_masks,labels=prep_input(sentences,labels, max_len=max_len)
# print('Original: ', sentences[0])
# print('Token IDs:', input_ids[0])

100%|██████████| 84916/84916 [05:02<00:00, 280.28it/s]


In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



#Validation split
tr_inputs, val_inputs, tr_labels, val_labels = train_test_split(input_ids, labels,stratify=labels,
                                                            random_state=2020, test_size=val_size)


tr_masks, val_masks, u,v =   train_test_split(attention_masks, labels,stratify=labels,
                                             random_state=2020, test_size=val_size)


train_dataset=TensorDataset(tr_inputs, tr_masks, tr_labels)
val_dataset=TensorDataset(val_inputs, val_masks, val_labels)
train_sampler = RandomSampler(train_dataset) 
valid_sampler = SequentialSampler(val_dataset)


# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = train_sampler, # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = valid_sampler, # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [16]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
num_classes = 27

In [18]:
class vec_output_FlaubertForSequenceClassification(FlaubertModel):
    
    config_class = FlaubertConfig
    

    def __init__(self, config):
        super().__init__(config)
        self.transformer = FlaubertModel(config)
        self.sequence_summary = SequenceSummary(config)
        self.init_weights()
        self.dropout =  torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(config.hidden_size, num_classes)


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        langs=None,
        token_type_ids=None,
        position_ids=None,
        lengths=None,
        cache=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        
        
        transformer_outputs = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            langs=langs,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            lengths=lengths,
            cache=cache,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        #output = self.dropout(output)
        output = transformer_outputs[0]
        vec = output[:,0]
        
        
        #logits
        dense = self.dropout(vec)
        
        #classifier
        logits = self.classifier(dense)
        
        outputs = (logits,) + transformer_outputs[1:]  # Keep new_mems and attention/hidden states if they are here
       
        
        return outputs,dense

In [19]:
len(Preprocess.dict_code_to_id)

27

In [20]:
modelname = 'flaubert-base-cased'

model = vec_output_FlaubertForSequenceClassification.from_pretrained(
        modelname, # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = len(Preprocess.dict_code_to_id), # The number of output labels--2 for binary classification.
        # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.cuda()

vec_output_FlaubertForSequenceClassification(
  (position_embeddings): Embedding(512, 768)
  (embeddings): Embedding(68729, 768, padding_idx=2)
  (layer_norm_emb): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (attentions): ModuleList(
    (0): MultiHeadAttention(
      (q_lin): Linear(in_features=768, out_features=768, bias=True)
      (k_lin): Linear(in_features=768, out_features=768, bias=True)
      (v_lin): Linear(in_features=768, out_features=768, bias=True)
      (out_lin): Linear(in_features=768, out_features=768, bias=True)
    )
    (1): MultiHeadAttention(
      (q_lin): Linear(in_features=768, out_features=768, bias=True)
      (k_lin): Linear(in_features=768, out_features=768, bias=True)
      (v_lin): Linear(in_features=768, out_features=768, bias=True)
      (out_lin): Linear(in_features=768, out_features=768, bias=True)
    )
    (2): MultiHeadAttention(
      (q_lin): Linear(in_features=768, out_features=768, bias=True)
      (k_lin): Linear(in_features=768, 

In [21]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [22]:
from transformers import get_linear_schedule_with_warmup
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 12

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [23]:
import torch.nn as nn
loss_criterion = nn.CrossEntropyLoss()


In [24]:
from sklearn.metrics import f1_score
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    
    #tr and val
    vec_output_tr = []
    vec_output_val =[]

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    best_f1 = 0
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        logits,vec = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask
                    )
        #new
        logits = logits[0]
        
        #Defining the loss
        loss = loss_criterion(logits, b_labels)
        
        #saving the features_tr
        vec = vec.detach().cpu().numpy()
        vec_output_tr.extend(vec)
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
        
        
        

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f} ".format(avg_train_loss))
    print("  Training epcoh took: {:} ".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    predictions=[]
    true_labels=[]
    

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits,vec = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask
                           )
            
        #new
        logits = logits[0]
        
        #defining the val loss
        loss = loss_criterion(logits, b_labels)
        
        
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Move logits and labels to CPU
        predicted_labels=np.argmax(logits,axis=1)
        predictions.extend(predicted_labels)
        label_ids = b_labels.to('cpu').numpy()
        true_labels.extend(label_ids)
        
        #saving the features_tr
        vec = vec.detach().cpu().numpy()
        vec_output_val.extend(vec)
        

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    print("Validation F1-Score: {}".format(f1_score(true_labels,predictions,average='macro')))
    curr_f1=f1_score(true_labels,predictions,average='macro')
    if curr_f1 > best_f1:
        best_f1=curr_f1
        torch.save(model.state_dict(), 'best_model.pt')
        np.save('best_vec_train.npy',vec_output_tr)
        np.save('best_vec_val.npy',vec_output_val)
    # Record all statistics from this epoch.
#     training_stats.append(
#         {
#             'epoch': epoch_i + 1,
#             'Training Loss': avg_train_loss,
#             'Valid. Loss': avg_val_loss,
#             'Valid. Accur.': avg_val_accuracy,
#             'Training Time': training_time,
#             'Validation Time': validation_time
#         }
#     )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 12 ========
Training...


/opt/conda/conda-bld/pytorch_1587428398394/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


  Batch    40  of  2,389.    Elapsed: 0:00:32.
  Batch    80  of  2,389.    Elapsed: 0:01:03.
  Batch   120  of  2,389.    Elapsed: 0:01:34.
  Batch   160  of  2,389.    Elapsed: 0:02:04.
  Batch   200  of  2,389.    Elapsed: 0:02:35.
  Batch   240  of  2,389.    Elapsed: 0:03:06.
  Batch   280  of  2,389.    Elapsed: 0:03:37.
  Batch   320  of  2,389.    Elapsed: 0:04:08.
  Batch   360  of  2,389.    Elapsed: 0:04:39.
  Batch   400  of  2,389.    Elapsed: 0:05:10.
  Batch   440  of  2,389.    Elapsed: 0:05:41.
  Batch   480  of  2,389.    Elapsed: 0:06:12.
  Batch   520  of  2,389.    Elapsed: 0:06:43.
  Batch   560  of  2,389.    Elapsed: 0:07:14.
  Batch   600  of  2,389.    Elapsed: 0:07:45.
  Batch   640  of  2,389.    Elapsed: 0:08:16.
  Batch   680  of  2,389.    Elapsed: 0:08:47.
  Batch   720  of  2,389.    Elapsed: 0:09:18.
  Batch   760  of  2,389.    Elapsed: 0:09:49.
  Batch   800  of  2,389.    Elapsed: 0:10:20.
  Batch   840  of  2,389.    Elapsed: 0:10:51.
  Batch   880

In [25]:
# Save model
# try:
#     model_state = {'model': model,
#               'state_dict': model.state_dict(),
#               'optimizer' : optimizer.state_dict()}

#     torch.save(model_state, 'saved_model.pth')
# except:
#     print('Error in saving model')

**Test model on unseen data**

# PREDICTIONS

**UNCOMMENT THE BELOW CELL IF TRAINING IS NOT PERFORMED IN THIS RUN**

In [26]:
model_path = '/kaggle/working/best_model.pt'

In [27]:
## Change the **model path** accordingly
# model_str = 'flaubert'
# model_path_dict = {'camembert':'/kaggle/input/camembertvinodh/saved_model.pth',
#                   'flaubert':'/kaggle/input/flaubertekansh/saved_model.pth'}

# model_path = model_path_dict[model_str]#'/kaggle/input/camembertvinodh/saved_model.pth'#'/kaggle/input/flaubertekansh/saved_model.pth'#
checkpoint = torch.load(model_path)
# model = checkpoint['model']
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [28]:
def predict_pyt(model, prediction_dataloader):
    """
    model: pytorch model
    prediction_dataloader: DataLoader object for which the predictions has to be made.
    return:
        predictions:- Direct predicted labels
        softmax_logits:- logits which are normalized with softmax on output"""
    
    # Put model in evaluation mode
    model.eval()

    # Tracking variables 
    predictions = []
    softmax_logits=[]
    vec_outputs = []
    
    # Predict 
    for batch in prediction_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        try:
            b_input_ids, b_input_mask = batch
        except ValueError:
            b_input_ids, b_input_mask, _ = batch
        # Telling the model not to compute or store gradients, saving memory and 
        # speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
            logits,vec = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask)
            
            logits = logits[0]

        
    #----- Add softmax---     
        m = nn.Softmax(dim=1)
    # #     input = torch.randn(2, 3)
        output = m(logits)
    #-------#------
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        predicted_labels=np.argmax(logits,axis=1)
        predictions.extend(predicted_labels)
        softmax_logits.extend(output)
        
        #vec_outputs saving
        vec = vec.detach().cpu().numpy()
        vec_outputs.extend(vec)

    print('DONE')
    return predictions, softmax_logits , vec_outputs

def predict_wrapper(model, sentences, max_len=max_len, batch_size = batch_size ):
    """
    Wrapper to create DataLoader object and predict, 
    this is if model and sentences are passed"""
    input_ids,attention_masks=prep_input(sentences,labels=None, max_len=max_len)
    prediction_data = TensorDataset(input_ids, attention_masks)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
    return predict_pyt(model, prediction_dataloader)

In [29]:
## Prepare the test dataset
batch_size = 32  

Preprocess.prepare_test(text_col)
test_sentences = Preprocess.test_sentences
X_test_phase1= Preprocess.X_test

In [30]:
# Predictions of validation set which is randomly separated from train dataset
start = time.time()
predictions, val_softmax_logits , vec_outputs= predict_pyt(model, validation_dataloader)
val_softmax_logits = np.array([ten.detach().cpu().numpy() for ten in val_softmax_logits])
np.save('validation_set_softmax_logits.npy',val_softmax_logits)
print('Time Taken Predict for val set: {:}'.format(format_time(time.time() - start)))

DONE
Time Taken Predict for val set: 0:01:04


In [31]:
## Predictions of test dataset 

start = time.time()
predictions, softmax_logits , vec_outputs = predict_wrapper(model, test_sentences)

#saving
np.save('best_vec_test.npy',vec_outputs)
softmax_logits = np.array([ten.detach().cpu().numpy() for ten in softmax_logits])
np.save('X_test_phase1_softmax_logits.npy',softmax_logits)
print('length of predictions {}'.format(len(predictions)))
print('Time Taken Predict for val set: {:}'.format(format_time(time.time() - start) ))

100%|██████████| 937/937 [00:03<00:00, 310.11it/s]


DONE
length of predictions 937
Time Taken Predict for val set: 0:00:10


In [32]:
X_test_phase1['prediction_model']= predictions
X_test_phase1['Prdtypecode']=X_test_phase1['prediction_model'].map(Preprocess.dict_id_to_code)
print(X_test_phase1['Prdtypecode'].value_counts())
X_test_phase1=X_test_phase1.drop(['prediction_model','Title','Description'],axis=1)

2583    115
2280     63
1280     57
1300     56
2060     55
2522     54
1560     53
1920     52
2403     50
1320     40
1160     37
10       35
1140     33
1302     28
40       28
1281     27
2585     24
2705     24
50       21
2582     20
2462     14
2905     11
1940      9
1301      9
60        8
1180      7
2220      7
Name: Prdtypecode, dtype: int64


In [33]:
X_test_phase1.to_csv('y_test_task1_phase1_pred.tsv',sep='\t',index=False)